In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import random
from feret_utils import get_feret_files_and_tags_dict, run_face_detection, create_dataset, create_dataset_gs
from matplotlib.pyplot import imshow
from IPython.display import Image
import cv2 as cv
import tensorflow as tf
import scipy
from datetime import datetime
import pickle
tfe = tf.contrib.eager

/home/michaelf/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Set tf basic settings
print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)
tf.enable_eager_execution()

1.9.0


In [3]:
import pickle
# Unpickle data_dict
with open('feret_data_dict_gs.pickle', 'rb') as handle:
    subject_list, data_dict, mean_image, std_image = pickle.load(handle)

In [4]:
print(data_dict['X_train'].shape)
print(data_dict['y_train'].shape)
print(data_dict['X_eval'].shape)
print(data_dict['y_eval'].shape)

(6931, 96, 96, 1)
(6931,)
(268, 96, 96, 1)
(268,)


In [5]:
# Create a classifier model which uses the data dict to create a multi loss optimizer which test 3 things
# A. classification of input 1
# B. classification of input 2
# C. test if imagess are of the same peron
class CombinedClassifier(tf.keras.Model):
    def __init__(self, num_of_ids):
        super().__init__()
        self.num_of_ids = num_of_ids
        
        # Conv layer 1 + Pooling
        self.conv1a = tf.keras.layers.Conv2D(filters=32,
                                            kernel_size=[4, 4],
                                            strides=(1, 1),
                                            padding='valid',
                                            activation=tf.nn.leaky_relu,
                                            use_bias=True,
                                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d()
                                           )
        
        self.pool1a = tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                               strides=(2, 2),
                                               padding='valid'
                                              )
        
        # Conv layer 2 + Pooling
        self.conv2a = tf.keras.layers.Conv2D(filters=32,
                                            kernel_size=[3, 3],
                                            strides=(1, 1),
                                            padding='same',
                                            activation=tf.nn.leaky_relu,
                                            use_bias=True,
                                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d()
                                           )
        
        self.pool2a = tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                               strides=(2, 2),
                                               padding='valid'
                                              )
        
        # Conv layer 3 + Pooling
        self.conv3a = tf.keras.layers.Conv2D(filters=32,
                                            kernel_size=[3, 3],
                                            strides=(1, 1),
                                            padding='same',
                                            activation=tf.nn.leaky_relu,
                                            use_bias=True,
                                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d()
                                           )
        
        self.pool3a = tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                               strides=(2, 2),
                                               padding='valid'
                                              )
        
        # Dense output layer
        self.fc1a = tf.keras.layers.Dense(16384, activation=tf.nn.relu)
        
        # Dropout layer
        self.dropout = tf.keras.layers.Dropout(rate=0.2)
        
        # Dense layer for classes
        self.fc2a = tf.keras.layers.Dense(num_of_ids)
        
        # Optimizers
        self.optimizer1 = tf.train.AdamOptimizer(learning_rate = 0.5*1e-4)
        self.optimizer2 = tf.train.AdamOptimizer(learning_rate = 1e-4)
        
    def call(self, inputs, training=True, **kwargs):
        half_batch_size = int(int(inputs.shape[0]) / 2)
        batch_size = int(half_batch_size * 2)
        
        # Input Layer
        input_layer = tf.reshape(inputs, [-1, 96, 96, 1])
        
        # Flow for first classifier
        x1 = self.conv1a(input_layer)
        x1 = self.pool1a(x1)
        x1 = self.conv2a(x1)
        x1 = self.pool2a(x1)
        x1 = self.conv3a(x1)
        x1 = self.pool3a(x1)
        x1 = tf.reshape(x1, [x1.shape[0], -1])
        x1_id_layer = self.fc1a(x1)
        if (training):
            x1_dropout = self.dropout(x1_id_layer)
            x1_logits = self.fc2a(x1_dropout)
            distances = tf.reduce_mean((x1_id_layer[0:half_batch_size] - x1_id_layer[half_batch_size:batch_size])**2, axis=1)

        else:
            x1_logits = self.fc2a(x1_id_layer)
            distances = 0

        
        return x1_logits, x1_id_layer, distances 
    
    def loss(self, logits1, labels1, distances, batch_size):        
        half_batch_size = int(batch_size / 2)
        # To make sure that both halves are equal
        batch_size = int(half_batch_size * 2)
        # Calculate losses according to classification requirments and comparison requirement
        # Loss 1: classification requirement
        onehot_labels = tf.one_hot(indices=tf.cast(labels1, tf.int32), depth = num_of_ids)
        loss_1 = tf.losses.softmax_cross_entropy(onehot_labels, logits1)
        
        # Loss 2: comparison requirement
        same = np.where(np.array(labels1[0:half_batch_size] - labels1[half_batch_size:batch_size]) == 0)[0]
        diff = np.arange(half_batch_size)
        diff = np.delete(diff, same)
        if (len(same) > 0):
            same_loss = tf.reduce_mean(tf.gather(distances, same)) * (len(same)/batch_size)
        else:
            same_loss = 0
        diff_loss = tf.reduce_mean(tf.maximum(0, 1 - tf.gather(distances, diff))) * (len(diff)/batch_size)
        loss_2 = (0.5 * same_loss) + (0.5 * diff_loss)
        return loss_1, loss_2
    
    def optimize(self, inputs, labels):
        with tf.GradientTape(persistent=True) as tape:
            x1_logits, x1_id_layer, distances = self(inputs)
            loss_1, loss_2 = self.loss(x1_logits, labels, distances, int(inputs.shape[0]))
        
        gradients = tape.gradient(loss_1, self.variables)
        self.optimizer1.apply_gradients(zip(gradients, self.variables))
        gradients = tape.gradient(loss_2, self.variables)
        self.optimizer2.apply_gradients(zip(gradients, self.variables))
        del(tape)
        return loss_1, loss_2
    
    def extract_features (self, inputs):
        x1_logits, x1_id_layer, distance = self(inputs)
        return x1_id_layer
    
    def test(self, inputs, labels, similarity_test=False):
        x1_logits, x1_id_layer, distance = self(inputs, training=False)
        test_class_1, test_class_2, test_compare = 0, 0, 0
        
        # Score of predecting the labels of the images
        pred_labels = tf.argmax(x1_logits, axis=-1)
        pred_labels = tf.cast(pred_labels, tf.float64)
        acc = tf.reduce_mean(tf.cast(tf.equal(pred_labels, labels), tf.float32))
        
        correct = 0
        incorrect = 0
        similarity = 0
        if (similarity_test):
            for i in range(inputs.shape[0] - 1):
                for j in range(i+1, inputs.shape[0]):
                    if labels[i] == labels[j]:
                        if (tf.reduce_mean(tf.abs((x1_id_layer[i] - x1_id_layer[j])**2))) < 1.5:
                            correct += 1
                        else:
                            incorrect += 1
                    else:
                        if (tf.reduce_mean(tf.abs((x1_id_layer[i] - x1_id_layer[j])**2))) > 1.5:
                            correct += 1
                        else:
                            incorrect += 1
            similarity = correct/(correct + incorrect)

        return acc, similarity
        


In [6]:
buffer_size = data_dict['X_train'].shape[0]
num_epochs = 300
batch_size = 500
train_ds = tf.data.Dataset.from_tensor_slices((data_dict['X_train'], data_dict['y_train']))
train_ds = train_ds.apply(tf.contrib.data.shuffle_and_repeat(buffer_size, num_epochs))
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.apply(tf.contrib.data.prefetch_to_device("/gpu:0"))

In [7]:
num_of_ids = len(subject_list)
#save = True
#load = True
#checkpoint_directory = "training_checkpoints/ckpt"

with tf.device('/gpu:0'):
    model = CombinedClassifier(num_of_ids=num_of_ids)
    for step, (batch_x, batch_y) in enumerate(train_ds):
        #if (load and step == 0):
        #    root = tfe.Checkpoint(model=model, optimizer_step=tf.train.get_or_create_global_step())
        #    root.restore(tf.train.latest_checkpoint(checkpoint_directory))
        #    print(model.variables)
        loss1, loss2 = model.optimize(batch_x, batch_y)   
        if (step % 10 == 0):
            print('[{0}] {1}'.format(datetime.now().strftime('%d.%m|%H:%M:%S'),
                                     "Step %d: loss1: %f loss2: %f" % (step, loss1, loss2)))
        if (step % 80 == 0):
            if (step % 240 == 0 and step != 0):
                similarity_test = True
            else:
                similarity_test = False
            accuracy, sim = model.test(data_dict['X_eval'], data_dict['y_eval'], similarity_test=similarity_test)
            print('[{0}] {1}'.format(datetime.now().strftime('%d.%m|%H:%M:%S'),
                                     "EVAL SET: Step %d: test accuracy: %f, sim accuracy: %f" % (step, accuracy, sim)))
        if (step % 40 == 0):
            accuracy, sim = model.test(batch_x, batch_y)
            print('[{0}] {1}'.format(datetime.now().strftime('%d.%m|%H:%M:%S'),
                                     "TRAIN SET: Step %d: test accuracy: %f, sim accuracy: %f" % (step, accuracy, sim)))
        #if (step % 20 == 0 and save and step > 0):
        #    root = tfe.Checkpoint(model=model, optimizer_step=tf.train.get_or_create_global_step())
        #    root.save(file_prefix=checkpoint_directory)
    


[21.08|08:05:42] Step 0: loss1: 6.552217 loss2: 0.248256
[21.08|08:05:42] EVAL SET: Step 0: test accuracy: 0.018657, sim accuracy: 0.000000
[21.08|08:05:42] TRAIN SET: Step 0: test accuracy: 0.030000, sim accuracy: 0.000000
[21.08|08:05:50] Step 10: loss1: 6.163083 loss2: 0.242632
[21.08|08:05:59] Step 20: loss1: 5.777877 loss2: 0.197173
[21.08|08:06:07] Step 30: loss1: 5.124956 loss2: 0.103773
[21.08|08:06:15] Step 40: loss1: 4.993493 loss2: 0.041574
[21.08|08:06:16] TRAIN SET: Step 40: test accuracy: 0.118000, sim accuracy: 0.000000
[21.08|08:06:24] Step 50: loss1: 4.787840 loss2: 0.016476
[21.08|08:06:32] Step 60: loss1: 4.492616 loss2: 0.018014
[21.08|08:06:41] Step 70: loss1: 4.015778 loss2: 0.010243
[21.08|08:06:49] Step 80: loss1: 3.747909 loss2: 0.003705
[21.08|08:06:49] EVAL SET: Step 80: test accuracy: 0.167910, sim accuracy: 0.000000
[21.08|08:06:49] TRAIN SET: Step 80: test accuracy: 0.290000, sim accuracy: 0.000000
[21.08|08:06:58] Step 90: loss1: 3.561287 loss2: 0.002445


[21.08|08:19:28] Step 890: loss1: 0.055192 loss2: 0.000490
[21.08|08:19:36] Step 900: loss1: 0.031770 loss2: 0.013774
[21.08|08:19:44] Step 910: loss1: 0.041161 loss2: 0.000601
[21.08|08:19:53] Step 920: loss1: 0.039341 loss2: 0.000220
[21.08|08:19:53] TRAIN SET: Step 920: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:20:01] Step 930: loss1: 0.022759 loss2: 0.000102
[21.08|08:20:10] Step 940: loss1: 0.030599 loss2: 0.000000
[21.08|08:20:18] Step 950: loss1: 0.014440 loss2: 0.000374
[21.08|08:20:26] Step 960: loss1: 0.017671 loss2: 0.002071
[21.08|08:20:52] EVAL SET: Step 960: test accuracy: 0.667910, sim accuracy: 0.984823
[21.08|08:20:52] TRAIN SET: Step 960: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:21:00] Step 970: loss1: 0.008756 loss2: 0.000331
[21.08|08:21:08] Step 980: loss1: 0.011615 loss2: 0.003487
[21.08|08:21:17] Step 990: loss1: 0.024983 loss2: 0.000208
[21.08|08:21:25] Step 1000: loss1: 0.024006 loss2: 0.001989
[21.08|08:21:25] TRAIN SET: Step 1

[21.08|08:33:46] Step 1790: loss1: 0.004543 loss2: 0.000102
[21.08|08:33:54] Step 1800: loss1: 0.005517 loss2: 0.000000
[21.08|08:33:55] TRAIN SET: Step 1800: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:34:03] Step 1810: loss1: 0.006424 loss2: 0.003205
[21.08|08:34:11] Step 1820: loss1: 0.005630 loss2: 0.002711
[21.08|08:34:20] Step 1830: loss1: 0.004190 loss2: 0.000150
[21.08|08:34:28] Step 1840: loss1: 0.007943 loss2: 0.008376
[21.08|08:34:29] EVAL SET: Step 1840: test accuracy: 0.664179, sim accuracy: 0.000000
[21.08|08:34:29] TRAIN SET: Step 1840: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:34:37] Step 1850: loss1: 0.006546 loss2: 0.000249
[21.08|08:34:45] Step 1860: loss1: 0.004465 loss2: 0.000000
[21.08|08:34:53] Step 1870: loss1: 0.005349 loss2: 0.002006
[21.08|08:35:02] Step 1880: loss1: 0.004033 loss2: 0.000000
[21.08|08:35:02] TRAIN SET: Step 1880: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:35:10] Step 1890: loss1: 0.003616 loss2: 0.

[21.08|08:47:55] Step 2680: loss1: 0.003097 loss2: 0.000997
[21.08|08:47:56] TRAIN SET: Step 2680: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:48:04] Step 2690: loss1: 0.002568 loss2: 0.000083
[21.08|08:48:12] Step 2700: loss1: 0.001806 loss2: 0.000000
[21.08|08:48:21] Step 2710: loss1: 0.003492 loss2: 0.002075
[21.08|08:48:29] Step 2720: loss1: 0.002896 loss2: 0.000104
[21.08|08:48:29] EVAL SET: Step 2720: test accuracy: 0.712687, sim accuracy: 0.000000
[21.08|08:48:29] TRAIN SET: Step 2720: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:48:37] Step 2730: loss1: 0.002270 loss2: 0.001923
[21.08|08:48:46] Step 2740: loss1: 0.003187 loss2: 0.000110
[21.08|08:48:54] Step 2750: loss1: 0.004531 loss2: 0.003161
[21.08|08:49:03] Step 2760: loss1: 0.003809 loss2: 0.000053
[21.08|08:49:03] TRAIN SET: Step 2760: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|08:49:11] Step 2770: loss1: 0.007712 loss2: 0.002216
[21.08|08:49:19] Step 2780: loss1: 0.005435 loss2: 0.

[21.08|09:04:02] Step 3710: loss1: 0.002771 loss2: 0.000287
[21.08|09:04:10] Step 3720: loss1: 0.001335 loss2: 0.000043
[21.08|09:04:10] TRAIN SET: Step 3720: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|09:04:18] Step 3730: loss1: 0.001031 loss2: 0.000451
[21.08|09:04:27] Step 3740: loss1: 0.000956 loss2: 0.000000
[21.08|09:04:35] Step 3750: loss1: 0.001268 loss2: 0.002881
[21.08|09:04:43] Step 3760: loss1: 0.001435 loss2: 0.000000
[21.08|09:04:44] EVAL SET: Step 3760: test accuracy: 0.723881, sim accuracy: 0.000000
[21.08|09:04:44] TRAIN SET: Step 3760: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|09:04:52] Step 3770: loss1: 0.001674 loss2: 0.000000
[21.08|09:05:00] Step 3780: loss1: 0.002160 loss2: 0.001393
[21.08|09:05:09] Step 3790: loss1: 0.002787 loss2: 0.001104
[21.08|09:05:17] Step 3800: loss1: 0.001363 loss2: 0.000000
[21.08|09:05:17] TRAIN SET: Step 3800: test accuracy: 1.000000, sim accuracy: 0.000000
[21.08|09:05:26] Step 3810: loss1: 0.002361 loss2: 0.

In [8]:
model.save_weights('checkpoints/my_checkpoint')

In [9]:
model_2 = CombinedClassifier(num_of_ids = num_of_ids)
model_2.load_weights('checkpoints/my_checkpoint')

In [10]:
accuracy, sim = model_2.test(data_dict['X_eval'], data_dict['y_eval'], similarity_test=True)
print(accuracy, sim)

tf.Tensor(0.73134327, shape=(), dtype=float32) 0.9422829671865393
